In [1]:
!nvidia-smi
!pip install ultralytics


Mon Nov  3 19:34:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ultralytics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
#copy data to a working directory
import shutil
#Copy files to working directory
inputPath = '/kaggle/input/warp-waste-recycling-plant-dataset/Warp-D'
workingPath = "/kaggle/working/waste_dataSet/Warp-D"
# Copy files to /kaggle/working/
shutil.copytree(inputPath, workingPath, dirs_exist_ok=True)

'/kaggle/working/waste_dataSet/Warp-D'

Create the yaml file

In [4]:
# Create train configuration
from pathlib import Path
import yaml
import random

workingPath = "/kaggle/working/waste_dataSet/Warp-D"

In [5]:
# read classes from classes.txt
with open( Path(workingPath).joinpath('classes.txt'), 'r') as f:
    classNames = [line.strip() for line in f.readlines()]
print(len(classNames))


28


**Split the train test**

In [6]:
# train and test split based on percentage
trainPath = Path(workingPath).joinpath('train/images')
trainAllFiles = [str(file) for file in trainPath.glob('*') if str(file).endswith(('.jpg', '.png'))]
random.shuffle(trainAllFiles)

trainTestSplit = 0.8
partitionPoint = int(trainTestSplit * len(trainAllFiles))
trainFiles = trainAllFiles[:partitionPoint]
validationFiles = trainAllFiles[partitionPoint:]

In [7]:
# Save file lists
trainFile = Path(workingPath).joinpath('train.txt')
valFile = Path(workingPath).joinpath('val.txt')
with open(trainFile, "w") as train_file:
    train_file.write("\n".join(trainFiles))
with open(valFile, "w") as val_file:
    val_file.write("\n".join(validationFiles))


In [8]:

data = {
    'train': str(trainFile),
    'val': str(valFile),
    'test' : str(Path(workingPath).joinpath('test/images')),
    'nc': len(classNames),
    'names': classNames
}

# dump yaml file 
yaml_output_path = Path(workingPath).joinpath('data.yaml')
with open(yaml_output_path, "w") as f:
    yaml.dump(data, f, default_flow_style=False)

# Train model

In [9]:
import ultralytics
ultralytics.checks()
!yolo train model=yolov8n.pt data=/kaggle/working/waste_dataSet/Warp-D/data.yaml epochs=50 freeze=5 imgsz=640

Ultralytics 8.3.224 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6515.7/8062.4 GB disk)
Ultralytics 8.3.224 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/waste_dataSet/Warp-D/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=5, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=Fa

# predict

In [10]:
!yolo predict model=/kaggle/working/yolov8n.pt source=/kaggle/working/waste_dataSet/Warp-D/test/images

Ultralytics 8.3.224 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

image 1/522 /kaggle/working/waste_dataSet/Warp-D/test/images/Monitoring_photo2_04-Mar_00-47-42.jpg: 384x640 (no detections), 50.6ms
image 2/522 /kaggle/working/waste_dataSet/Warp-D/test/images/Monitoring_photo_04-Mar_08-36-50.jpg: 384x640 (no detections), 6.6ms
image 3/522 /kaggle/working/waste_dataSet/Warp-D/test/images/Monitoring_photo_2_test_25-Mar_11-09-46.jpg: 384x640 (no detections), 7.0ms
image 4/522 /kaggle/working/waste_dataSet/Warp-D/test/images/Monitoring_photo_2_test_25-Mar_11-11-15.jpg: 384x640 (no detections), 6.6ms
image 5/522 /kaggle/working/waste_dataSet/Warp-D/test/images/Monitoring_photo_2_test_25-Mar_11-11-23.jpg: 384x640 (no detections), 6.5ms
image 6/522 /kaggle/working/waste_dataSet/Warp-D/test/images/Monitoring_photo_2_test_25-Mar_11-12-02.jpg: 384x640 (no detections), 6.4ms
image 7/522 /kaggle/working/wa

In [11]:
# # zip file for download
# import shutil

# # Define the folder and output zip file path
# folder_path = '/kaggle/working/runs/detect'  # Replace with the folder you want to zip
# zip_path = '/kaggle/working/runs/detect.zip'  # Path where the zip will be saved

# # Zip the folder
# shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

In [12]:
# # Save file lists
# trainFile = Path(workingPath).joinpath('train.txt')
# valFile = Path(workingPath).joinpath('val.txt')
# with open(trainFile, "w") as train_file:
#     train_file.write("\n".join(trainFiles))
# with open(valFile, "w") as val_file:
#     val_file.write("\n".join(validationFiles))

In [13]:
# data = {
#     'train': str(trainFile),
#     'val': str(valFile),
#     'test' : str(Path(workingPath).joinpath('test/images')),
#     'nc': len(classNames),
#     'names': classNames
# }

# # dump yaml file 
# yaml_output_path = Path(workingPath).joinpath('data.yaml')
# with open(yaml_output_path, "w") as f:
#     yaml.dump(data, f, default_flow_style=False)